In [47]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
# Display up to 500 columns of a wide dataframe, instead of ...-ing them 
pd.set_option('display.max_columns', 500)

# Assignment
In this assignment, you'll be working with the U.S. Education Dataset from Kaggle. The data gives detailed state level information on several facets of the state of education on an annual basis. To learn more about the data and the column descriptions, click the Kaggle link above.

Access this data from the Thinkful database using the following credentials:

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

In [23]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

# I connected via pgAdmin4 and verified the table name is the same as the db name
used_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

## To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

### 1. Determine all the variable types and find the fraction of the missing values for each variable.

PRIMARY_KEY, STATE are the only two non-numeric fields.

State is categorical, nominal data--mostly the full names of a state, district, or US territory. There are a few outlying values that are 2-3 character abbreviations, and a few variations containing "DOD" we can probably combine, if we don't want to drop these rows altogether.

YEAR is technically continuous, interval, but its use here is probably more categorical, ordinal--we're not using year to _measure_ a length of time, but sorting by the YEAR column allows us to compare scores year over year.

PRIMARY_KEY is a combination of 4-digit year and state|district|territory. I think having the year as a part of this key makes it categorical, ordinal, for the same reasons that YEAR alone is categorical, ordinal.

Revenues and expenditures are dollar amounts. These values are continuous, ratio.

ENROLL and GRADES_* are the numbers of students in each grade, or in a range of grades (this comes directly from [the Kaggle page](https://www.kaggle.com/noriuk/us-education-datasets-unification-project/home)). These fields are continuous, ratio.

The AVG_* scores represent a state's average score for fourth or eighth graders taking an NAEP exam in reading or math. I found some information about interpreting NAEP scores [here](https://nces.ed.gov/nationsreportcard/mathematics/interpret_results.aspx). These values are continuous, interval--there is a "zero" within the possible range of values, but the ranges are not the same between grades, and it's not clear that an average score of, say, 300 means those students know exactly twice as much about reading or math as a group of students with an average of 150.

In [5]:
used_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

You can see from the counts above that there are null values in most of the columns outside of the key/state/year. We can get the percentage of null values per column like this:

In [6]:
# The sum of null rows for each column * 100, divided by the total number of rows for each column:
used_df.isnull().sum()*100/used_df.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   14.209115
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.595174
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

The AVG_* scores in particular have a high percentage of missing values.

In [7]:
used_df.loc[used_df['AVG_MATH_4_SCORE'].notnull()]

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN
5,1992_COLORADO,COLORADO,1992,NaN,3185173.0,163253.0,1307986.0,1713934.0,3264826.0,1642466.0,...,50648.0,45025.0,34533.0,394904.0,160299.0,562613.0,221.023429,272.398433,213.480890,266.419687
6,1992_CONNECTICUT,CONNECTICUT,1992,NaN,3834302.0,143542.0,1342539.0,2348221.0,3721338.0,2148041.0,...,38058.0,33691.0,28366.0,304284.0,126917.0,436932.0,226.798480,273.739345,222.417599,264.713904
7,1992_DELAWARE,DELAWARE,1992,NaN,645233.0,45945.0,420942.0,178346.0,638784.0,372722.0,...,8272.0,8012.0,6129.0,67495.0,28338.0,96296.0,217.899972,262.868585,206.389052,261.577921
8,1992_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1992,NaN,709480.0,64749.0,0.0,644731.0,742893.0,329160.0,...,5832.0,5000.0,3433.0,47009.0,18173.0,70000.0,192.600553,234.924621,178.557612,265.976765
9,1992_FLORIDA,FLORIDA,1992,NaN,11506299.0,788420.0,5683949.0,5033930.0,11305642.0,5166374.0,...,164416.0,142372.0,100835.0,1276685.0,511557.0,1819706.0,213.689751,259.908848,204.882606,262.118986
10,1992_GEORGIA,GEORGIA,1992,NaN,5536901.0,398701.0,2798674.0,2339526.0,5535942.0,3043984.0,...,99648.0,92374.0,63646.0,787599.0,315539.0,1106441.0,215.594906,259.359434,206.844175,NaN


### 2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

__For a lot of these columns, it is probably fine to fill in nulls with a mean or median value:__
* ENROLL - the number of students per state probably doesn't tend to jump or drop a lot year over year.
* REVENUE and EXPENDITURE - Same here, I can't see these columns changing too significantly.
* GRADES - A mean number of students for each state can be used to fill in missing values in these columns.
* AVG_* - This column is already a mean value, so why not calculate and use a mean to fill in missing rows.

### 3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

* ENROLL / GRADES_ALL_G - Where one is populated and the other is not for the same year, use the populated value. The explanation on the Kaggle page suggested these two numbers are estimates of student counts, just calculated by different agencies. 
* REVENUE columns - I spot-checked a few states and these columns seem to either fully populated for a given year, or all NaN. I don't know how we would populate these by only looking at other columns in the same row.
* EXPENDITURE columns - I saw a few where OTHER_EXPENDITURE was NaN, but otherwise this seems to be a similar situation to the revenue columns--either fully populated, or all NaN.
   
    I thought that for rows where OTHER_EXPENDITURE isn't populated, we could estimate its value by subtracting all of the known expenditures from TOTAL. That technique doesn't seem to give me the same value for OTHER, where that column is populated. (You can see one of my spot checks below.)
    
* GRADES columns - Seem to be either fully populated for a given year, or all NaN. I'm not sure how we would replace these when missing if we are only looking at other columns in the same row, except for the GRADES_ALL_G column I already discussed above.
* AVG score columns - There is no other data in a given row that would be useful in estimating these values, if they are not populated.

In [25]:
used_df.loc[used_df['STATE'] == 'NEW_YORK']#[['STATE','YEAR','ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']]

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
32,1992_NEW_YORK,NEW_YORK,1992,NaN,21630179.0,1172030.0,8697663.0,11760486.0,22815611.0,13288972.0,6132532.0,NaN,1640971.0,31290.0,201674.0,197698.0,184815.0,142822.0,1585404.0,727840.0,2344534.0,218.445535,266.422300,211.740921,NaN
85,1993_NEW_YORK,NEW_YORK,1993,2663637.0,22688753.0,1276362.0,8848470.0,12563921.0,23934282.0,14029639.0,6425626.0,935895.0,1625848.0,31687.0,205238.0,198259.0,188274.0,143055.0,1608489.0,743933.0,2384109.0,NaN,NaN,NaN,NaN
137,1994_NEW_YORK,NEW_YORK,1994,2702937.0,23933662.0,1442038.0,9092275.0,13399349.0,25359726.0,14829280.0,6782725.0,992378.0,1717307.0,32227.0,212239.0,202024.0,191181.0,144139.0,1627846.0,746836.0,2406909.0,NaN,NaN,NaN,NaN
190,1995_NEW_YORK,NEW_YORK,1995,2738361.0,25063999.0,1368074.0,9932669.0,13763256.0,26540967.0,15693535.0,6970608.0,945437.0,1811288.0,32087.0,216112.0,203089.0,194347.0,142841.0,1653846.0,761509.0,2447442.0,NaN,NaN,NaN,NaN
241,1996_NEW_YORK,NEW_YORK,1996,2788343.0,26026366.0,1510447.0,10284155.0,14231764.0,26974514.0,15973623.0,6953428.0,942356.0,1835923.0,31566.0,210795.0,205572.0,196724.0,146738.0,1677128.0,769382.0,2478076.0,222.634059,270.231486,215.779216,NaN
292,1997_NEW_YORK,NEW_YORK,1997,2818497.0,26775074.0,1446691.0,10477889.0,14850494.0,28227561.0,16259479.0,7136050.0,933478.0,2561130.0,32086.0,206548.0,210379.0,197148.0,146818.0,1691688.0,776218.0,2499992.0,NaN,NaN,NaN,NaN
344,1998_NEW_YORK,NEW_YORK,1998,2834992.0,27900803.0,1522836.0,11033327.0,15344640.0,29853891.0,17266937.0,7384638.0,1044857.0,2723181.0,38196.0,202894.0,214629.0,200097.0,148315.0,1705145.0,774469.0,2517810.0,NaN,NaN,NaN,NaN
396,1999_NEW_YORK,NEW_YORK,1999,2850352.0,30215370.0,1794853.0,12605931.0,15814586.0,31841870.0,18323434.0,7913623.0,893364.0,3309820.0,37613.0,199271.0,219305.0,202221.0,150444.0,1717122.0,781175.0,2535910.0,NaN,NaN,NaN,NaN
449,2000_NEW_YORK,NEW_YORK,2000,2861478.0,31906526.0,1938590.0,13766358.0,16201578.0,33683043.0,19856637.0,8377510.0,1057876.0,3266115.0,39062.0,194673.0,217881.0,203482.0,151043.0,1718478.0,782021.0,2539561.0,226.559061,276.262859,222.445055,NaN
501,2001_NEW_YORK,NEW_YORK,2001,2856277.0,34373567.0,1912495.0,16032781.0,16428291.0,38062603.0,22052316.0,9100764.0,1157116.0,4579196.0,40212.0,190402.0,214283.0,210369.0,153505.0,1717579.0,790657.0,2548448.0,NaN,NaN,NaN,NaN


In [45]:
used_df_ny_1993 = used_df.loc[(used_df['STATE'] == 'FLORIDA') & (used_df['YEAR'] == 1993)]
used_df_ny_1993

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
61,1993_FLORIDA,FLORIDA,1993,1980569.0,11370156.0,947326.0,5520893.0,4901937.0,11425183.0,5333136.0,3557043.0,852220.0,1424334.0,34793.0,167140.0,168509.0,149054.0,100646.0,1313261.0,525569.0,1873623.0,NaN,NaN,NaN,NaN


In [46]:
calc_other_expenditure = int(used_df_ny_1993['TOTAL_EXPENDITURE'] - used_df_ny_1993['INSTRUCTION_EXPENDITURE'] - used_df_ny_1993['SUPPORT_SERVICES_EXPENDITURE'] - used_df_ny_1993['CAPITAL_OUTLAY_EXPENDITURE'])
print("Calculated Other Expenditure: {}".format(calc_other_expenditure))
other_expenditure = int(used_df_ny_1993['OTHER_EXPENDITURE'])
diff_expenditure = calc_other_expenditure - other_expenditure
print("Other Expenditure: {}\nDifference: {}".format(other_expenditure,diff_expenditure))

Calculated Other Expenditure: 1110670
Other Expenditure: 852220
Difference: 258450


### 4. This time, fill in the missing values using interpolation (extrapolation).


* ENROLL - Calculate the mean of the surrounding populated years. If the first/last values within the range of years are not populated, either use the next or last available value, or calculate a mean based on the nearest 2-5 populated years. The first/last issue seems to be a consistent one across all of the states I spot checked.

* REVENUE columns - Where these columns are all NaN, calculate the mean for each column for, say, the previous 2-5 years. This way we're still using numbers that are relatively close in time, measured in dollars whose value has not been changed significantly by inflation (probably NOT true when comparing early 1990s dollars to late 2010s).

* EXPENDITURE columns - We can treat these expenditure columns the same way as the revenue columns, including rows where just OTHER_EXPENDITURE isn't populated.

* GRADES columns - These columns can also be treated like the revenue columns if they're not populated. I see the two most recent years are the ones that are not populated, which means we're probably using the same interpolated mean for both?

* AVG_* - The population pattern of the AVG columns implies that the NAEP tests aren't administered every year, or that data is only collected for them periodically. I also see some cases where not all 4 averages are available for a given year.

    Between years where a set of values is available, I'd either carry the previous values forward, or calculate the mean of the previous and next new values. For instance, for 1993-1995, we could carry forward values from 1992, or get the mean of the 1992 and 1996 average scores, and use that.

### 5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

In general, the more rows of data we consider, the more options we have for filling in missing values. Taking year into consideration made things more complex but is probably also going to yield interpolated values that are a better estimate than simply getting the mean of all available values.